In [147]:
#pip install pykml
#pip install geopandas
#pip install pystac_client
#pip install rioxarray

In [1]:
import pandas as pd
from pyproj import Proj, transform
import numpy as np
from typing import Tuple
import numpy as np
import geopandas as gpd
from pykml.factory import KML_ElementMaker as KML
import geopandas as gpd
import fiona
import rioxarray
from PIL import Image

In [4]:
# Load GeoJSON file, this is the high voltage network in part of Italy
file_path = "rete_elettrodotti_app.geojson"
gdf = gpd.read_file(file_path)
gdf.head()

,gml_id,dgc_codice,codice,topon,dimens,geometry
0,rete_elettrodotti.0,6711.0,0403,NaN,1548.97,"LINESTRING (1731199.25000 4831341.50000, 17312..."
1,rete_elettrodotti.1,6704.0,0403,NaN,3007.65,"LINESTRING (1731419.02000 4829505.37000, 17313..."
2,rete_elettrodotti.2,6705.0,0403,NaN,255.58,"LINESTRING (1732318.76000 4831380.28000, 17323..."
3,rete_elettrodotti.3,6715.0,0403,NaN,5419.39,"LINESTRING (1732406.44000 4830388.73000, 17324..."
4,rete_elettrodotti.4,6703.0,0403,NaN,804.52,"LINESTRING (1732429.31000 4831149.91000, 17324..."


In [5]:
#def linestring_to_points(feature,line):
#    return {feature:line.coords}

#gdf['points'] = gdf.apply(lambda l: linestring_to_points(l['codice'], l['geometry']), axis=1)

In [6]:
#geoms = gdf.geometry

# Display the first geometry in the GeoDataFrame
#print(geoms[0])

In [7]:
def get_coords(geom):
    return geom.coords[0]

# Apply the function to the geometry column
gdf['coordinates_origin'] = gdf['geometry'].apply(get_coords)

In [8]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
df = gpd.read_file('rete_elettrodotti_app.kml', driver='KML')
df.head()

,Name,Description,geometry
0,,,"LINESTRING (11.86405 43.59904, 11.86417 43.598..."
1,,,"LINESTRING (11.86598 43.58246, 11.86522 43.582..."
2,,,"LINESTRING (11.87792 43.59904, 11.87805 43.598..."
3,,,"LINESTRING (11.87858 43.59009, 11.87855 43.589..."
4,,,"LINESTRING (11.87919 43.59693, 11.87987 43.595..."
...,...,...,...
85,,,"LINESTRING (11.78349 43.29903, 11.78272 43.299..."
86,,,"LINESTRING (11.87583 43.34381, 11.86993 43.34903)"
87,,,"LINESTRING (11.87614 43.34387, 11.87023 43.34903)"
88,,,"LINESTRING (11.91574 43.30915, 11.91401 43.310..."


In [9]:
def get_coords(geom):
    return geom.coords[0]

# Apply the function to the geometry column
df['coordinates'] =df['geometry'].apply(get_coords)

In [11]:
location=[]
for i in df['geometry']:
    location.append(list(i.coords))

In [12]:
lon=[]
lat=[]
for i in location:
    for j in i:
        lon.append(j[0])
        lat.append(j[1])

In [13]:
# find the central area of the high voltage network to query the image from 
min_lon=min(lon)
max_lon=max(lon)
min_lat=min(lat)
max_lat=max(lat)

In [55]:
print((min_lon + max_lon)/2)

11.87232651372281

In [56]:
print((min_lat + max_lat)/2)

43.44903099168371

In [31]:
api_url = "https://earth-search.aws.element84.com/v0"
from pystac_client import Client

client = Client.open(api_url)

In [32]:
collection = "sentinel-s2-l2a-cogs"  # Sentinel-2, Level 2A, COGs

In [61]:
from shapely.geometry import Point
point = Point(11.87, 43.45)  
#round(11.87232651372281,2)
#round(43.44903099168371,2)

In [71]:
search = client.search(
    collections=[collection],
    intersects=point,
    max_items=10,
)

In [72]:
print(search.matched())

785


In [73]:
items = search.get_all_items()

In [135]:
item = items[3]
print(item.datetime)
print(item.geometry)
print(item.properties)

2023-03-16 10:08:46+00:00
{'type': 'Polygon', 'coordinates': [[[12.81261993814928, 43.20122074964178], [11.549770306028183, 43.23637418759707], [11.872524455947243, 44.217278157543255], [12.875725873123068, 44.18761302539064], [12.81261993814928, 43.20122074964178]]]}
{'datetime': '2023-03-16T10:08:46Z', 'platform': 'sentinel-2b', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'gsd': 10, 'view:off_nadir': 0, 'proj:epsg': 32632, 'sentinel:utm_zone': 32, 'sentinel:latitude_band': 'T', 'sentinel:grid_square': 'QP', 'sentinel:sequence': '0', 'sentinel:product_id': 'S2B_MSIL2A_20230316T100029_N0509_R122_T32TQP_20230316T130117', 'sentinel:data_coverage': 83.16, 'eo:cloud_cover': 46.85, 'sentinel:valid_cloud_cover': True, 'sentinel:processing_baseline': '05.09', 'sentinel:boa_offset_applied': True, 'created': '2023-03-16T20:22:28.294Z', 'updated': '2023-03-16T20:22:28.294Z'}


In [131]:
assets = items[3].assets  # first item's asset dictionary
# number 3 is fine, with less cloud
print(assets.keys())

dict_keys(['thumbnail', 'overview', 'info', 'metadata', 'visual', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL'])


In [132]:
for key, asset in assets.items():
    print(f"{key}: {asset.title}")

thumbnail: Thumbnail
overview: True color image
info: Original JSON metadata
metadata: Original XML metadata
visual: True color image
B01: Band 1 (coastal)
B02: Band 2 (blue)
B03: Band 3 (green)
B04: Band 4 (red)
B05: Band 5
B06: Band 6
B07: Band 7
B08: Band 8 (nir)
B8A: Band 8A
B09: Band 9
B11: Band 11 (swir16)
B12: Band 12 (swir22)
AOT: Aerosol Optical Thickness (AOT)
WVP: Water Vapour (WVP)
SCL: Scene Classification Map (SCL)


In [102]:
print(assets["thumbnail"].href)

https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/32/T/QP/2023/3/16/0/preview.jpg


In [133]:
b08_href = assets["B8A"].href
b08 = rioxarray.open_rasterio(b08_href)
b08.rio.to_raster("B08.tif")

b02_href = assets["B02"].href
b02 = rioxarray.open_rasterio(b02_href)
b02.rio.to_raster("B02.tif")

b03_href = assets["B03"].href
b03 = rioxarray.open_rasterio(b03_href)
b03.rio.to_raster("B03.tif")

b04_href = assets["B04"].href
b04 = rioxarray.open_rasterio(b04_href)
b04.rio.to_raster("B04.tif")

<xarray.DataArray (band: 1, y: 5490, x: 5490)>
[30140100 values with dtype=uint16]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 7e+05 7e+05 7e+05 ... 8.097e+05 8.097e+05 8.098e+05
  * y            (y) float64 4.9e+06 4.9e+06 4.9e+06 ... 4.79e+06 4.79e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  AVERAGE
    _FillValue:          0
    scale_factor:        1.0
    add_offset:          0.0


In [125]:
im08 = Image.open('B08.tif')
im08_array = numpy.array(im08)
im08_array
#im08.show()

array([[   0,    0,    0, ...,    1,    1,    1],
       [   0,    0,    0, ...,    1,    1,    1],
       [   0,    0,    0, ...,    1,    1,    1],
       ...,
       [   0,    0,    0, ..., 2705, 2957, 2767],
       [   0,    0,    0, ..., 2697, 2842, 2482],
       [   0,    0,    0, ..., 2598, 2622, 2234]], dtype=uint16)

In [126]:
im04 = Image.open('B04.tif')
im04_array = numpy.array(im04)
im04_array

/Users/wentingjiang/opt/anaconda3/lib/python3.9/site-packages/PIL/Image.py:3035: DecompressionBombWarning: Image size (120560400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


array([[   0,    0,    0, ...,    1,    1,    1],
       [   0,    0,    0, ...,    1,    1,   10],
       [   0,    0,    0, ...,    1,    1,    1],
       ...,
       [   0,    0,    0, ..., 1246,  976,  990],
       [   0,    0,    0, ..., 1130, 1019,  866],
       [   0,    0,    0, ...,  903,  944,  972]], dtype=uint16)

In [148]:
im04_array.shape
#need to down sample the 2,3,4 bands

(10980, 10980)

In [129]:
im08_array.shape

(5490, 5490)

In [139]:
def downsample_by_averaging(img: np.ndarray, window_shape: Tuple[int, int]) -> np.ndarray:
    return np.mean(
        img.reshape((
            *img.shape[:-2],
            img.shape[-2] // window_shape[-2], window_shape[-2],
            img.shape[-1] // window_shape[-1], window_shape[-1],
        )),
        axis=(-1, -3),
    )

im04_array_new=downsample_by_averaging(im04_array, (2, 2))
im04_array_new

array([[0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        1.00000e+00, 3.25000e+00],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        1.00000e+00, 1.00000e+00],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        1.00000e+00, 1.00000e+00],
       ...,
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.28450e+03,
        1.48100e+03, 1.19725e+03],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.29350e+03,
        1.38800e+03, 1.05050e+03],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.04700e+03,
        1.10825e+03, 9.50250e+02]])

In [144]:
#(NIR-RED) / (NIR+RED) -> vegetation, adding a small number to avoid NaNs
ndvi=(im08_array-im04_array_new)/(im08_array+im04_array_new+0.0001)
ndvi

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -0.52939931],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.35605965,
         0.33258224,  0.39597653],
       [ 0.        ,  0.        ,  0.        , ...,  0.3517103 ,
         0.34373522,  0.40523707],
       [ 0.        ,  0.        ,  0.        , ...,  0.42551439,
         0.40580389,  0.40315615]])

In [146]:
img = Image.fromarray(ndvi, 'RGB')
#img.save('my.png')
img.show()

In [149]:
#Normalized Difference Moisture Index (NDMI)
#Sentinel-2 NDMI = (B08 - B11) / (B08 + B11)

b11_href = assets["B11"].href
b11 = rioxarray.open_rasterio(b11_href)
b11.rio.to_raster("B11.tif")

In [150]:
im11 = Image.open('B11.tif')
im11_array = numpy.array(im11)
im11_array

array([[   0,    0,    0, ...,   22,   28,   34],
       [   0,    0,    0, ...,   19,   24,   32],
       [   0,    0,    0, ...,   31,   27,   26],
       ...,
       [   0,    0,    0, ..., 3015, 3370, 3094],
       [   0,    0,    0, ..., 3014, 3150, 2826],
       [   0,    0,    0, ..., 2739, 2809, 2466]], dtype=uint16)

In [156]:
NDMI = (im08_array - im11_array) / (im08_array + im11_array + +0.0001)
NDMI

array([[   0.        ,    0.        ,    0.        , ..., 2848.46587624,
        2258.92324509, 1871.50893855],
       [   0.        ,    0.        ,    0.        , ..., 3275.88362058,
        2620.50951796, 1984.99398487],
       [   0.        ,    0.        ,    0.        , ..., 2047.05610295,
        2339.63450131, 2426.32434695],
       ...,
       [   0.        ,    0.        ,    0.        , ...,   11.40314665,
          10.29287166,   11.12591689],
       [   0.        ,    0.        ,    0.        , ...,   11.41989124,
          10.88584762,   12.2818385 ],
       [   0.        ,    0.        ,    0.        , ...,   12.25313824,
          12.03259046,   13.89446779]])